# Projeto: Assistente Virtual com PLN em Python

Este notebook implementa um sistema de assistente virtual do zero, cobrindo os seguintes módulos:
1.  **Text-to-Speech (TTS):** Para que a assistente possa falar.
2.  **Speech-to-Text (STT):** Para que a assistente possa ouvir e entender comandos de voz.
3.  **Lógica de Comandos:** Para executar ações com base nos comandos recebidos.

### Fase 1: Instalação das Bibliotecas

Primeiro, instalamos todas as dependências necessárias para o projeto.

In [ ]:
!pip install SpeechRecognition gTTS playsound wikipedia-api geopy

**Nota Importante sobre `PyAudio`:**
A biblioteca `SpeechRecognition` precisa do `PyAudio` para acessar o microfone. A instalação via `pip` pode ser complicada. Se a célula acima ou as próximas derem erro relacionado ao PyAudio:

* **No Anaconda:** Tente `conda install -c anaconda pyaudio` em seu terminal.
* **No Linux (Debian/Ubuntu):** Tente `sudo apt-get install python3-pyaudio`.
* **No Google Colab:** Geralmente funciona, mas se não, pode ser necessário procurar por workarounds específicos.

### Fase 2: Módulo de Fala (Text-to-Speech)

Esta função será a "voz" da nossa assistente. Ela recebe um texto, converte para um arquivo de áudio `.mp3` usando a API do Google Text-to-Speech e o reproduz.

In [ ]:
from gtts import gTTS
from playsound import playsound
import os

def falar(texto):
    """
    Converte um texto em áudio e o reproduz.
    """
    try:
        # Cria o objeto gTTS com o texto em português do Brasil
        tts = gTTS(text=texto, lang='pt-br')
        
        # Salva o áudio em um arquivo temporário
        nome_arquivo = 'resposta_temp.mp3'
        tts.save(nome_arquivo)
        
        # Imprime o que a assistente vai dizer e reproduz o áudio
        print(f"Assistente: {texto}")
        playsound(nome_arquivo)
        
        # Remove o arquivo de áudio temporário após a reprodução
        os.remove(nome_arquivo)
    except Exception as e:
        print(f"Ocorreu um erro na síntese de voz: {e}")

# Teste rápido da função (descomente a linha abaixo para testar)
# falar("Olá, meu módulo de fala está funcionando.")

### Fase 3: Módulo de Audição (Speech-to-Text)

Esta função será o "ouvido" da assistente. Ela usa o microfone para capturar a fala do usuário e a biblioteca `SpeechRecognition` para transcrevê-la em texto.

In [ ]:
import speech_recognition as sr

def ouvir_comando():
    """
    Captura o áudio do microfone e o transcreve para texto.
    """
    r = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Ouvindo...")
        # Ajusta o reconhecedor para o ruído do ambiente para melhorar a precisão
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        
    try:
        print("Reconhecendo...")
        # Usa a API de reconhecimento de fala do Google
        comando = r.recognize_google(audio, language='pt-BR')
        print(f"Usuário disse: {comando}")
        # Retorna o comando em letras minúsculas para facilitar o processamento
        return comando.lower()
    except sr.UnknownValueError:
        # Erro caso a fala não seja entendida
        print("Não foi possível entender o áudio.")
        return None
    except sr.RequestError as e:
        # Erro caso o serviço do Google esteja indisponível
        print(f"Não foi possível solicitar resultados do serviço; {e}")
        return None

### Fase 4: Módulo de Ações

Aqui definimos as funções específicas que a assistente pode executar, como pesquisar na Wikipedia, abrir o YouTube ou encontrar uma farmácia.

In [ ]:
import webbrowser
import wikipediaapi
from datetime import datetime

# Configura a API da Wikipedia para buscar artigos em português
wiki_pt = wikipediaapi.Wikipedia('AssistenteVirtual/1.0', 'pt')

def pesquisar_wikipedia(termo):
    """
    Pesquisa um termo na Wikipedia e resume o resultado.
    """
    falar(f"Buscando por {termo} na Wikipedia...")
    pagina = wiki_pt.page(termo)
    
    if pagina.exists():
        # Pega os primeiros 250 caracteres do resumo para uma resposta concisa
        resumo = pagina.summary[:250]
        falar(f"De acordo com a Wikipedia, {resumo}...")
    else:
        falar(f"Desculpe, não encontrei resultados para {termo} na Wikipedia.")

def abrir_youtube():
    """
    Abre o site do YouTube no navegador padrão.
    """
    falar("Abrindo o YouTube...")
    webbrowser.open("https://www.youtube.com")

def encontrar_farmacia_proxima():
    """
    Pergunta por uma localização e abre uma busca no Google Maps.
    """
    falar("Claro. Perto de qual endereço ou ponto de referência?")
    local = ouvir_comando()
    if local:
        falar(f"Ok, buscando por farmácias perto de {local}.")
        # Cria uma URL de busca no Google Maps e a abre no navegador
        url_busca = f"https://www.google.com/maps/search/farmacias+perto+de+{local.replace(' ', '+')}"
        webbrowser.open(url_busca)
    else:
        falar("Não consegui entender a localização. Por favor, tente novamente.")
        
def que_horas_sao():
    """
    Informa a hora atual.
    """
    agora = datetime.now()
    falar(f"Agora são {agora.hour} horas e {agora.minute} minutos.")

### Fase 5: Loop Principal (O Cérebro da Assistente)

Esta é a parte final que une tudo. O loop principal fica aguardando um comando de voz e, com base em palavras-chave, decide qual ação tomar. Para parar a assistente, basta dizer "encerrar" ou "parar".

In [ ]:
def executar_assistente():
    """
    Loop principal que escuta e responde aos comandos.
    """
    falar("Assistente virtual online. Como posso ajudar?")
    
    while True:
        comando = ouvir_comando()
        
        # Se um comando for reconhecido, processe-o
        if comando:
            # Comando para pesquisar na Wikipedia
            if 'wikipédia sobre' in comando or 'pesquise sobre' in comando:
                termo = comando.replace('wikipédia sobre', '').replace('pesquise sobre', '').strip()
                if termo:
                    pesquisar_wikipedia(termo)
                else:
                    falar("Por favor, diga o que você quer que eu pesquise.")
            
            # Comando para abrir o YouTube
            elif 'youtube' in comando:
                abrir_youtube()
            
            # Comando para encontrar farmácias
            elif 'farmácia' in comando:
                encontrar_farmacia_proxima()

            # Comando para saber as horas
            elif 'que horas são' in comando:
                que_horas_sao()
            
            # Comando para encerrar a assistente
            elif 'encerrar' in comando or 'parar' in comando:
                falar("Encerrando. Até logo!")
                break
            
            # Caso o comando não seja reconhecido
            else:
                falar("Desculpe, não entendi o comando. Pode repetir?")

### --- INICIAR A ASSISTENTE ---
Para iniciar a assistente, execute a célula de código abaixo. Ela irá chamar a função `executar_assistente` e começar o loop de audição.

In [ ]:
# Chama a função principal para iniciar a assistente
# O programa ficará em loop aqui até que você diga "encerrar" ou "parar".
executar_assistente()